# DerivKit: DerivativeKit — Tabulated derivatives (single point, noisy table)

### Summary
This notebook demonstrates how to estimate a **first derivative** from **noisy tabulated data** using
`DerivativeKit` in *tabulated mode*. We use a simple toy problem where the underlying function is
$\sin(x)$ (so the truth derivative is $\cos(x)$), add Gaussian noise to the sampled table (to
emulate a realistic scenario where tabulated data is noisy), and then
estimate $f'(x_0)$ at a single point using two methods:

- **finite differences + Ridders extrapolation**
- **adaptive local fit**

### Usage
If you prefer to run the same example as a script:

```bash
$ python demo-scripts/08-derivative-kit-tabulated-simple.py
```

### What it does
- Builds a noisy tabulated dataset with a fixed random seed (reproducible).
- Sets a single evaluation point $x_0 = 0.7$.
- Computes $f'(x_0)$ from the noisy table using two methods.
- Prints the two estimates, their absolute errors, and a simple “winner”.

### Notes
- The “truth” comparison uses the **noise-free** analytic derivative $\cos(x_0)$.
- With noise, estimates will vary with the random seed and noise level.

### Requirements
- `derivkit` installed and importable in your Python environment.

---
### Other derivatives (dx/dy, log-slopes)

By default, `Tabulated1DModel(x_tab, y_tab)` represents a function $y=f(x)$, so `DerivativeKit`
computes **$dy/dx$**.

If you want a different derivative, re-parameterize the tabulated data:

- **$dx/dy$**
  Build the model with swapped axes, i.e. tabulate $x=g(y)$ by using `Tabulated1DModel(y_tab, x_tab)`
  and evaluate the derivative at a chosen value of $y_0$.
  **Important:** this assumes $y$ is (locally) monotonic so the inverse makes sense; in practice,
  sort by $y$ before building the model.

- **Log–log slope $d\log y / d\log x$**
  Work in log space by tabulating $\log y$ as a function of $\log x$, i.e.
  `Tabulated1DModel(np.log(x_tab), np.log(y_tab))`.

- **Inverse log–log slope $d\log x / d\log y$**
  Swap the log-transformed arrays (again requiring monotonicity in $\log y$), i.e.
  `Tabulated1DModel(np.log(y_tab), np.log(x_tab))`.

In all cases, `Tabulated1DModel` accepts your arrays directly, so you can load real data
(e.g. via `np.loadtxt` or `np.load`) and choose the parametrization that matches the derivative
you want to estimate.



In [ ]:
import sys
from pathlib import Path

import numpy as np

# Make repo root importable so this notebook works when run from derivkit-demos/
# (Adjust the parents[...] level if your notebook lives somewhere else.)
sys.path.insert(0, str(Path.cwd().resolve().parents[0]))

from derivkit.derivative_kit import DerivativeKit
from derivkit.tabulated_model.one_d import Tabulated1DModel


# 1) Build (or load) x_tab, y_tab
rng = np.random.default_rng(42)

n_tab = 70  # number of tabulated points
x_tab = np.linspace(0.0, 2.0 * np.pi, n_tab)  # x values on a regular grid

y_noise_sigma = 0.05  # gaussian noise level in y_tab, set to 5% of signal amplitude
y_tab = np.sin(x_tab) + rng.normal(0.0, y_noise_sigma, size=x_tab.shape)  # noisy sin(x) samples

# If your x/y come from a file, replace the block above with e.g.:
# data = np.loadtxt("my_data.txt")  # two columns
# x_tab = data[:, 0]
# y_tab = data[:, 1]
#
# Or:
# x_tab = np.load("x.npy")
# y_tab = np.load("y.npy")

model = Tabulated1DModel(x_tab, y_tab, extrapolate=True)


# 2) Differentiate at a single point
x0 = 0.7
truth = float(np.cos(x0))  # analytic derivative for the sin(x) toy example

dk = DerivativeKit(function=model, x0=x0)

d_fr = float(
    np.asarray(dk.differentiate(method="finite", order=1, extrapolation="ridders"))
    .reshape(-1)[0]
)
d_ad = float(
    np.asarray(dk.differentiate(method="adaptive", order=1, n_points=27, spacing=0.25))
    .reshape(-1)[0]
)

e_fr = abs(d_fr - truth)
e_ad = abs(d_ad - truth)


# 3) Print comparison
print("DerivKit tabulated derivative demo (single point)")
print(f"x0 = {x0:.3f}")
print(f"noise sigma in y_tab: {y_noise_sigma:g}  (seed=42)")
print()
print(f"truth  f'(x0)=cos(x0)      = {truth:+.6f}")
print(f"finite (Ridders) estimate  = {d_fr:+.6f}   |err| = {e_fr:.3e}")
print(f"adaptive estimate          = {d_ad:+.6f}   |err| = {e_ad:.3e}")
print(f"winner (smaller |err|): {'adaptive' if e_ad < e_fr else 'finite (Ridders)'}")


# 4) What to try next (notes)
print("\nNext steps:")
print("- Change `y_noise_sigma` or the RNG seed to see how estimates vary with noise.")
print("- Replace `x_tab, y_tab` with your own arrays (loaded from a file) to run on real data.")
print("  Tabulated1DModel accepts x and y arrays directly; load them (np.loadtxt / np.load) and pass them in.")
